In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pysal.explore import inequality
from inequality.gini import Gini_Spatial
from pysal.lib import weights
import warnings
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

C:\Users\bartb\miniconda3\envs\ugsa\Lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [20]:
ls_in = ['Abidjan','Addis Ababa','Baghdad','Bangalore','Belgrade','Belo Horizonte','Bogota','Bologna','Cali','Canberra',\
         'Casablanca','Chandigarh','Chelyabinsk','Chengdu','Christchurch','Cochabamba','Copenhagen','Curitiba','Damascus',\
         'Detroit','Dhaka','Dongguan','Fortaleza','Ghent','Guatemala City','Gwangju','Hanoi','Indore','Johannesburg',\
         'Kampala','Karlsruhe','Kathmandu','Kharkiv','Kuala Lumpur','Kumamoto','Lagos','Leicester','Liverpool',\
         'Lubumbashi','Lviv','Manila','Memphis','Mexico City','Mombasa','Nanjing','New Delhi','Oran','Oslo','Perth',\
         'Philadelphia','Pune','Rennes','Rosario','Rostov','San Antonio','San Diego','Santo Domingo','Seattle','Seoul',\
         'Sevilla','Shijiazhuang','Shiraz','Skopje','Surabaya','Tallinn','Tashkent','Tel Aviv','Toronto','Turin',\
         'Utrecht','Utsunomiya','Washington DC','Yaounde','Yerevan','Zibo']

In [3]:
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(ls_in)]
cities_adj = cities_adj.sort_values('City').reset_index()

ls_adj = ['Canberra','Karlsruhe','Oslo']

cities_adj2 = cities[cities['City'].isin(ls_adj)]
cities_adj2 = cities_adj2.sort_values('City').reset_index()

In [51]:
ls_out

array(['Abidjan.gpkg', 'Addis Ababa.gpkg', 'Baghdad.gpkg',
       'Bangalore.gpkg', 'Belgrade.gpkg', 'Belo Horizonte.gpkg',
       'Bogota.gpkg', 'Bologna.gpkg', 'Cali.gpkg', 'Casablanca.gpkg',
       'Chandigarh.gpkg', 'Chelyabinsk.gpkg', 'Chengdu.gpkg',
       'Christchurch.gpkg', 'Cochabamba.gpkg', 'Copenhagen.gpkg',
       'Curitiba.gpkg', 'Damascus.gpkg', 'Detroit.gpkg', 'Dhaka.gpkg',
       'Dongguan.gpkg', 'Fortaleza.gpkg', 'Ghent.gpkg',
       'Guatemala City.gpkg', 'Gwangju.gpkg', 'Hanoi.gpkg', 'Indore.gpkg',
       'Johannesburg.gpkg', 'Kampala.gpkg', 'Kathmandu.gpkg',
       'Kharkiv.gpkg', 'Kuala Lumpur.gpkg', 'Kumamoto.gpkg', 'Lagos.gpkg',
       'Leicester.gpkg', 'Liverpool.gpkg', 'Lubumbashi.gpkg', 'Lviv.gpkg',
       'Manila.gpkg', 'Memphis.gpkg', 'Mexico City.gpkg', 'Mombasa.gpkg',
       'Nanjing.gpkg', 'New Delhi.gpkg', 'Oran.gpkg', 'Perth.gpkg',
       'Philadelphia.gpkg', 'Pune.gpkg', 'Rennes.gpkg', 'Rosario.gpkg',
       'Rostov.gpkg', 'San Antonio.gpkg', 'San Die

In [ ]:
%%time
path = "D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/Grid_geoms/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('.')[-1] == 'gpkg' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]
ls_bool = [any([j in i for j in ls_adj]) for i in ls_file]

ls_sub = np.array(ls_file)[ls_bool]
ls_out = np.array(ls_file)[~np.array(ls_bool)]

area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]

j = 0
k = 0
city_score = []
for i in range(len(ls_file)):
    if ls_bool[i]:
        city_score.append(gpd.read_file(path+ls_sub[j]))
        
        print(ls_sub[j].split('_')[-1].split('.')[0])
        j = j+1
    else:
        city_score.append(gpd.read_file(path+ls_out[k]))
        
        print(ls_out[k].split('_')[-1].split('.')[0])
        k = k+1

#city_score = [gpd.read_file(path+i) for i in ls_file]

Abidjan
Addis Ababa


In [ ]:
%%time
# For loop city/GINI
# First non-spatial GINI for the 50 cities.

# Queen contignuity
# Spatial GINI - value cell itself
# Weight is values of surrounding cells
# Near and far, with weights of 1 for near and 0 for far.

warnings.filterwarnings('ignore')
cities_gini = []
cities_spatialgini = []
for city in enumerate(city_score):
    tr_gini = []
    tr_spatialgini = []
    print(ls_file[city[0]].split('.')[0])
    for i in city[1].columns[city[1].columns.str.contains('Sc-access')]:
        float_df = city[1][['geometry','population',i]]

        gini = pd.DataFrame(float_df[i].value_counts()).sort_index()
        gini['pop'] = float_df.groupby(i)['population'].sum()
        gini = gini.reset_index()
        gini['frac_pop'] = gini['pop'] / gini['pop'].sum()
        gini['richer_frac'] = 1-gini['frac_pop'].cumsum()
        gini['ugs'] = (gini['index'] * gini['pop']) 
        gini['frac_ugs'] = gini.ugs/gini.ugs.sum()
        gini_score = 1-(gini.frac_ugs*(gini.frac_pop+2*gini.richer_frac)).sum()
        print('gini',i.rsplit(' ')[-1],round(gini_score,4))
        tr_gini.append(gini_score)
        
        wq = weights.Queen.from_dataframe(float_df)
        wq.transform = "B"
        spatial_gini = Gini_Spatial(float_df[i], wq).g
        print('spatial gini',i.rsplit(' ')[-1],round(spatial_gini,4))
        tr_spatialgini.append(spatial_gini)
    cities_gini.append(tr_gini)
    cities_spatialgini.append(tr_spatialgini)
    print(' ')

In [ ]:
gini_df = pd.DataFrame(cities_gini, columns = ['Gini E-300','Gini E-600','Gini E-1000'])
spatialgini_df = pd.DataFrame(cities_spatialgini, columns = ['S-gini E-300','S-gini E-600','S-gini E-1000'])
comb_gini = pd.concat([gini_df, spatialgini_df], axis = 1)
comb_gini.index = [i.split('.')[0] for i in ls_file]
comb_gini

In [ ]:
comb_gini2 = comb_gini
comb_gini.sort_values(by='S-gini E-300')

In [ ]:
comb_gini.to_csv('D:/Dumps/GEE-WP Scores/E2SFCA/gini_scores.csv')

In [ ]:
%%time
path = "D:/Dumps/GEE-WP Scores/Gravity/100m grids/Grid_geoms/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('_')[1] == 'grav3' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_in]) for i in ls_file]]
ls_bool = [any([j in i for j in ls_adj]) for i in ls_file]

ls_sub = np.array(ls_file)[ls_bool]
ls_out = np.array(ls_file)[~np.array(ls_bool)]

area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]

j = 0
k = 0
city_score = []
for i in range(len(ls_file)):
    if ls_bool[i]:
        city_score.append(gpd.read_file(path+ls_sub[j]))
        
        print(ls_sub[j].split('_')[-1].split('.')[0])
        j = j+1
    else:
        city_score.append(gpd.read_file(path+ls_out[k]))
        
        print(ls_out[k].split('_')[-1].split('.')[0])
        k = k+1

In [ ]:
%%time
warnings.filterwarnings('ignore')
Gcities_gini = []
Gcities_spatialgini = []
for city in enumerate(city_score):
    Gtr_gini = []
    Gtr_spatialgini = []
    print(ls_file[city[0]].split('.')[0].split('_')[-1])
    for i in city[1][['tr_300','tr_600','tr_1000']]:
        float_df = city[1][city[1][i].isna() == False][['geometry','population',i]]

        gini = pd.DataFrame(float_df[i].value_counts()).sort_index()
        gini['pop'] = float_df.groupby(i)['population'].sum()
        gini = gini.reset_index()
        gini['frac_pop'] = gini['pop'] / gini['pop'].sum()
        gini['richer_frac'] = 1-gini['frac_pop'].cumsum()
        gini['ugs'] = (gini['index'] * gini['pop']) 
        gini['frac_ugs'] = gini.ugs/gini.ugs.sum()
        gini_score = 1-(gini.frac_ugs*(gini.frac_pop+2*gini.richer_frac)).sum()
        print('gini',i.rsplit('_')[-1],round(gini_score,4))
        Gtr_gini.append(gini_score)
        
        wq = weights.Queen.from_dataframe(float_df)
        wq.transform = "B"
        spatial_gini = Gini_Spatial(float_df[i], wq).g
        print('spatial gini',i.rsplit('_')[-1],round(spatial_gini,4))
        Gtr_spatialgini.append(spatial_gini)
    Gcities_gini.append(Gtr_gini)
    Gcities_spatialgini.append(Gtr_spatialgini)
    print(' ')

In [ ]:
Ggini_df = pd.DataFrame(Gcities_gini, columns = ['Gini G-300','Gini G-600','Gini G-1000'])
Gspatialgini_df = pd.DataFrame(Gcities_spatialgini, columns = ['S-gini G-300','S-gini G-600','S-gini G-1000'])
Gcomb_gini = pd.concat([Ggini_df, Gspatialgini_df], axis = 1)
Gcomb_gini.index = [i.split('.')[0].split('_')[-1] for i in ls_file]
Gcomb_gini

In [ ]:
Gcomb_gini.to_csv('D:/Dumps/GEE-WP Scores/Gravity/gini_scores')

In [ ]:
gini_comb = pd.concat([comb_gini, Gcomb_gini], axis = 1)
gini_comb.to_csv('D:/Dumps/GEE-WP Scores/gini_all.csv')
gini_comb